In [46]:
from web3 import Web3
from web3.middleware import geth_poa_middleware

w3 = Web3(Web3.HTTPProvider('https://rinkeby.infura.io/v3/08ef2304798243e6b4dd03945d85e2ac'))

In [47]:
w3.middleware_onion.clear()
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.clientVersion

'Geth/v1.10.13-omnibus-abb549b5/linux-amd64/go1.17.3'

In [48]:
contract_address='0xc3CD1fE9b0848fF9929F1e2b1879f255231cc89F'
ABI = [
	{
		"inputs": [
			{
				"internalType": "contract IERC20",
				"name": "Token",
				"type": "address"
			}
		],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"inputs": [],
		"name": "FaucetAmount",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "UserTimeStamp",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "UserToClaim",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "claim",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "isOwner",
		"outputs": [
			{
				"internalType": "bool",
				"name": "",
				"type": "bool"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "amount",
				"type": "uint256"
			}
		],
		"name": "lock",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "token",
		"outputs": [
			{
				"internalType": "contract IERC20",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]


In [49]:
Faucet_instance = w3.eth.contract(contract_address, abi=ABI)

In [101]:
#Get testing Account
#This part would not happen in backend
private_key = "2edb0f009f6a5118cd50e155d3dff0d611479c5717b288877608a0f949ea356a"
acct = w3.eth.account.privateKeyToAccount(private_key)

nonce = w3.eth.get_transaction_count('0x830219365648553C0972543B7ace77403c8E0E26')  

# Claim Token

In [102]:
#Build TX
tx_hash = Faucet_instance.functions.claim().buildTransaction({
     'chainId': 4,
     'gas': 70000,
     'maxFeePerGas': w3.toWei('2', 'gwei'),
     'maxPriorityFeePerGas': w3.toWei('1', 'gwei'),
     'nonce': nonce,
 })
tx_hash

{'value': 0,
 'chainId': 4,
 'gas': 70000,
 'maxFeePerGas': 2000000000,
 'maxPriorityFeePerGas': 1000000000,
 'nonce': 3,
 'to': '0xc3CD1fE9b0848fF9929F1e2b1879f255231cc89F',
 'data': '0x4e71d92d'}

In [103]:
#Sign tx
signed_txn = w3.eth.account.sign_transaction(tx_hash, private_key=private_key)

In [98]:
#Send tx
w3.eth.send_raw_transaction(signed_txn.rawTransaction)  

HexBytes('0xba142055d4fc52683b2df593293b674c9deaedf22afccc02e9f906e59364e922')

# View Functions

In [104]:
#View remaining CST in Faucet
Faucet_instance.functions.FaucetAmount().call()

99999999998998

In [135]:
#View user next claim time

def timeToClaim():
    totalSecs = Faucet_instance.functions.UserToClaim().call()

    if(totalSecs > 86400):
        print("You haven't claimed yet")
    else:
        hours = int(totalSecs/3600)
        minutes = int((totalSecs - hours*3600)/60)
        seconds = totalSecs%60
        print("Your can claim again in", hours, "hours", minutes, "minutes", seconds, "seconds")

timeToClaim()

Your can claim again in 4 hours 59 minutes 1 seconds


# Test CST amount of my address

In [126]:
CSTabi =[
        {
            "inputs": [
                {
                    "internalType": "string",
                    "name": "name",
                    "type": "string"
                },
                {
                    "internalType": "string",
                    "name": "symbol",
                    "type": "string"
                }
            ],
            "stateMutability": "nonpayable",
            "type": "constructor"
        },
        {
            "anonymous": "false",
            "inputs": [
                {
                    "indexed": "true",
                    "internalType": "address",
                    "name": "owner",
                    "type": "address"
                },
                {
                    "indexed": "true",
                    "internalType": "address",
                    "name": "spender",
                    "type": "address"
                },
                {
                    "indexed": "false",
                    "internalType": "uint256",
                    "name": "value",
                    "type": "uint256"
                }
            ],
            "name": "Approval",
            "type": "event"
        },
        {
            "anonymous": "false",
            "inputs": [
                {
                    "indexed": "true",
                    "internalType": "address",
                    "name": "from",
                    "type": "address"
                },
                {
                    "indexed": "true",
                    "internalType": "address",
                    "name": "to",
                    "type": "address"
                },
                {
                    "indexed": "false",
                    "internalType": "uint256",
                    "name": "value",
                    "type": "uint256"
                }
            ],
            "name": "Transfer",
            "type": "event"
        },
        {
            "inputs": [
                {
                    "internalType": "address",
                    "name": "owner",
                    "type": "address"
                },
                {
                    "internalType": "address",
                    "name": "spender",
                    "type": "address"
                }
            ],
            "name": "allowance",
            "outputs": [
                {
                    "internalType": "uint256",
                    "name": "",
                    "type": "uint256"
                }
            ],
            "stateMutability": "view",
            "type": "function"
        },
        {
            "inputs": [
                {
                    "internalType": "address",
                    "name": "spender",
                    "type": "address"
                },
                {
                    "internalType": "uint256",
                    "name": "amount",
                    "type": "uint256"
                }
            ],
            "name": "approve",
            "outputs": [
                {
                    "internalType": "bool",
                    "name": "",
                    "type": "bool"
                }
            ],
            "stateMutability": "nonpayable",
            "type": "function"
        },
        {
            "inputs": [
                {
                    "internalType": "address",
                    "name": "account",
                    "type": "address"
                }
            ],
            "name": "balanceOf",
            "outputs": [
                {
                    "internalType": "uint256",
                    "name": "",
                    "type": "uint256"
                }
            ],
            "stateMutability": "view",
            "type": "function"
        },
        {
            "inputs": [],
            "name": "decimals",
            "outputs": [
                {
                    "internalType": "uint8",
                    "name": "",
                    "type": "uint8"
                }
            ],
            "stateMutability": "view",
            "type": "function"
        },
        {
            "inputs": [
                {
                    "internalType": "address",
                    "name": "spender",
                    "type": "address"
                },
                {
                    "internalType": "uint256",
                    "name": "subtractedValue",
                    "type": "uint256"
                }
            ],
            "name": "decreaseAllowance",
            "outputs": [
                {
                    "internalType": "bool",
                    "name": "",
                    "type": "bool"
                }
            ],
            "stateMutability": "nonpayable",
            "type": "function"
        },
        {
            "inputs": [
                {
                    "internalType": "address",
                    "name": "spender",
                    "type": "address"
                },
                {
                    "internalType": "uint256",
                    "name": "addedValue",
                    "type": "uint256"
                }
            ],
            "name": "increaseAllowance",
            "outputs": [
                {
                    "internalType": "bool",
                    "name": "",
                    "type": "bool"
                }
            ],
            "stateMutability": "nonpayable",
            "type": "function"
        },
        {
            "inputs": [],
            "name": "name",
            "outputs": [
                {
                    "internalType": "string",
                    "name": "",
                    "type": "string"
                }
            ],
            "stateMutability": "view",
            "type": "function"
        },
        {
            "inputs": [],
            "name": "symbol",
            "outputs": [
                {
                    "internalType": "string",
                    "name": "",
                    "type": "string"
                }
            ],
            "stateMutability": "view",
            "type": "function"
        },
        {
            "inputs": [],
            "name": "totalSupply",
            "outputs": [
                {
                    "internalType": "uint256",
                    "name": "",
                    "type": "uint256"
                }
            ],
            "stateMutability": "view",
            "type": "function"
        },
        {
            "inputs": [
                {
                    "internalType": "address",
                    "name": "recipient",
                    "type": "address"
                },
                {
                    "internalType": "uint256",
                    "name": "amount",
                    "type": "uint256"
                }
            ],
            "name": "transfer",
            "outputs": [
                {
                    "internalType": "bool",
                    "name": "",
                    "type": "bool"
                }
            ],
            "stateMutability": "nonpayable",
            "type": "function"
        },
        {
            "inputs": [
                {
                    "internalType": "address",
                    "name": "sender",
                    "type": "address"
                },
                {
                    "internalType": "address",
                    "name": "recipient",
                    "type": "address"
                },
                {
                    "internalType": "uint256",
                    "name": "amount",
                    "type": "uint256"
                }
            ],
            "name": "transferFrom",
            "outputs": [
                {
                    "internalType": "bool",
                    "name": "",
                    "type": "bool"
                }
            ],
            "stateMutability": "nonpayable",
            "type": "function"
        },
        {
            "inputs": [
                {
                    "internalType": "address",
                    "name": "account",
                    "type": "address"
                },
                {
                    "internalType": "uint256",
                    "name": "amount",
                    "type": "uint256"
                }
            ],
            "name": "mint",
            "outputs": [],
            "stateMutability": "nonpayable",
            "type": "function"
        }
    ]

In [127]:
CST_addr= '0xb41E0D7463F11D3e772C8B3aD3b9bea6DAb03D62'

In [128]:
CST_instance = w3.eth.contract(CST_addr, abi=CSTabi)

In [134]:
CST_instance.functions.balanceOf('0x830219365648553C0972543B7ace77403c8E0E26').call()

1002